#Predicting sentiment from product reviews

#Fire up GraphLab Create

In [1]:
import graphlab

#Read some product review data

Loading reviews for a set of baby products. 

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to sdhar.anlyt@gmail.comand will expire on October 05, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-1916 - Server binary: C:\Users\sumit.dhar\AppData\Local\Dato\Dato Launcher\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\SUMIT~1.DHA\AppData\Local\Temp\graphlab_server_1445046539.log.0
[INFO] GraphLab Server Version: 1.6.1


#Let's explore this data together

Data includes the product name, the review text and the rating of the review. 

In [3]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


#Build the word count vector for each review

In [3]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [4]:
products.head()

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5L, 'stink': 1L,'because': 1L, 'order ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 2L, 'highly': 1L, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1L, 'and':3L, 'love': 2L, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'parents!!':1L, 'all': 2L, 'puppe ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'doll': 1L, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'it': 1L, ' ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'feeding,': 1L, 'and':2L, 'all': 1L, 'right': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'help': 1L,'give': 1L, 'is': 1L, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1L, 'all':1L, 'standarad': 1L, ..."


In [5]:
graphlab.canvas.set_target('ipynb')

In [6]:
products['name'].show()

#Examining the reviews for most-sold product:  'Vulli Sophie the Giraffe Teether'

In [7]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [8]:
len(giraffe_reviews)

785

In [9]:
giraffe_reviews['rating'].show(view='Categorical')

#Build a sentiment classifier

In [11]:
products['rating'].show(view='Categorical')

##Define what's a positive and a negative sentiment

We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment.  Reviews with a rating of 4 or higher will be considered positive, while the ones with rating of 2 or lower will have a negative sentiment.   

In [12]:
#ignore all 3* reviews
products = products[products['rating'] != 3]

In [10]:
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'] >=4

In [11]:
products.head()

name,review,rating,word_count,sentiment
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5L, 'stink': 1L,'because': 1L, 'order ...",0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 2L, 'highly': 1L, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1L, 'and':3L, 'love': 2L, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'parents!!':1L, 'all': 2L, 'puppe ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'doll': 1L, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'it': 1L, ' ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'feeding,': 1L, 'and':2L, 'all': 1L, 'right': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'help': 1L,'give': 1L, 'is': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1L, 'all':1L, 'standarad': 1L, ...",1


##Let's train the sentiment classifier

In [12]:
train_data,test_data = products.random_split(.8, seed=0)

In [13]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 146861
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 237351
PROGRESS: Number of coefficients    : 237352
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 6        | 0.000001  | 2.555051     | 0.764655          | 0.763703            |
PROGRESS: | 2         | 9        | 5.000000  | 3.510073     | 0.895575          | 0.826779            |
PROGRESS: | 3         | 10       | 5.000000  | 3.983082     | 0.91

#Evaluate the sentiment model

In [14]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+----------------+------------------+-------+------+
 |    threshold     |      fpr       |       tpr        |   p   |  n   |
 +------------------+----------------+------------------+-------+------+
 |       0.0        | 0.185625288417 | 0.00927015367062 | 28047 | 8668 |
 | 0.0010000000475  | 0.814374711583 |  0.990729846329  | 28047 | 8668 |
 | 0.00200000009499 | 0.785302261191 |  0.988590580098  | 28047 | 8668 |
 | 0.00300000002608 | 0.764305491463 |  0.986772203801  | 28047 | 8668 |
 | 0.00400000018999 | 0.747231195201 |  0.985559952936  | 28047 | 8668 |
 | 0.00499999988824 | 0.734771573604 |  0.984383356509  | 28047 | 8668 |
 | 0.00600000005215 | 0.722773419474 |  0.983812885514  | 28047 | 8668 |
 | 0.00700000021607 | 0.714120904476 |  0.983028487895  | 28047 | 8668 |
 | 0.00800000037998 | 0.705006922012 |  0.982137126965  | 28047 | 8668 |
 | 0.00899999961257 

In [15]:
sentiment_model.show(view='Evaluation')

#Applying the learned model to understand sentiment for Giraffe

In [16]:
giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews, output_type='probability')

In [17]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'and': 1L, 'all': 1L,'because': 1L, 'it': 1L, ...",0.997713976229
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'and': 1L, 'right': 1L,'help': 1L, 'just': 1L, ...",0.998305995832
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'and': 2L, 'all': 1L,'latex.': 1L, 'being': ...",1.24423729415e-05
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'and': 2L, 'one!': 1L,'all': 1L, 'love': 1L, ...",0.995124709002
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'and': 3L, 'all': 1L,'old': 1L, 'her.': 1L, ...",0.783333367343
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0,"{'and': 6L, 'seven': 1L,'already': 1L, 'love': ...",0.999992656691
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0,"{'and': 4L, 'drooling,':1L, 'love': 1L, 'her.': ...",0.980155846791
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0,"{'and': 3L, 'chew': 1L,""don't"": 1L, 'is': 1L, ...",0.999628625614
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0,"{'chew': 2L, 'because':1L, 'just': 2L, 'what': ...",0.957081628067
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0,"{'and': 2L, 'already':1L, 'in': 1L, 'some': ...",0.000870330258935


##Sort the reviews based on the predicted sentiment and explore

In [18]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [19]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5.0,"{'giggles': 1L, 'all':1L, ""violet's"": 2L, ...",1.0
Vulli Sophie the GiraffeTeether ...,My nephews and my fourkids all had Sophie in ...,5.0,"{'and': 4L, 'chew': 1L,'all': 1L, 'perfect;': ...",0.999999995946
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5.0,"{'all': 2L, ""don't"": 1L,'(literally).so': 1L, ...",0.999999994752
Vulli Sophie the GiraffeTeether ...,So much more than ateether. It helps my ...,5.0,"{'and': 4L, 'chew': 1L,'all': 1L, 'since': 1L, ...",0.999999979126
Vulli Sophie the GiraffeTeether ...,Sophie la giraffe isabsolutely the best toy ...,5.0,"{'and': 5L, 'the': 1L,'all': 1L, 'old': 1L, ...",0.999999976154
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4.0,"{'all': 2L, 'pops': 1L,'existence.': 1L, ...",0.999999945084
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4.0,"{'peace': 1L, 'month':1L, 'bright': 1L, ...",0.999999937466
Vulli Sophie the GiraffeTeether ...,Never thought I'd see myson French kissing a ...,5.0,"{'giggles': 1L, 'all':1L, 'out,': 1L, 'over': ...",0.999999890791
Vulli Sophie the GiraffeTeether ...,My 5-mos old son took tothis immediately. The ...,5.0,"{'just': 1L, 'shape': 2L,'mutt': 1L, '""dog': 1L, ...",0.999999887912
Vulli Sophie the GiraffeTeether ...,Sophie the Giraffe is theperfect teething toy. ...,5.0,"{'just': 2L, 'both': 1L,'month': 1L, 'ears,': ...",0.999999880634


##Most positive reviews for the giraffe

In [25]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [26]:
giraffe_reviews[1]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

##Show most negative reviews for giraffe

In [27]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

In [28]:
giraffe_reviews[-2]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."

# Sentiment Analysis Assignment

In [20]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [21]:
selected_words

['awesome',
 'great',
 'fantastic',
 'amazing',
 'love',
 'horrible',
 'bad',
 'terrible',
 'awful',
 'wow',
 'hate']

In [7]:
products[‘awesome’] = products[‘word_count’].apply(awesome_count)

SyntaxError: invalid syntax (<ipython-input-7-6fc22396d249>, line 1)

In [22]:
products['awesome']= products['word_count'].apply(lambda wc: wc.get("awesome", 0))

In [23]:
products['awesome'].sum()

2090L

In [24]:
for word in selected_words:
    products[word] = products['word_count'].apply(lambda wc: wc.get(word, 0))
    print word
    print products[word].sum()

awesome
2090
great
45206.0
fantastic
932.0
amazing
1363.0
love
42065.0
horrible
734
bad
3724.0
terrible
748
awful
383
wow
144
hate
1220


In [52]:
train_data,test_data = products.random_split(.8, seed=0)

In [26]:
features=selected_words

In [69]:
len(test_data)

36670

In [67]:
test_data['sentiment'].sum()

28005L

### 76%

In [53]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=selected_words,
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 146861
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 11
PROGRESS: Number of unpacked features : 11
PROGRESS: Number of coefficients    : 12
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 0.187007     | 0.768182          | 0.767903            |
PROGRESS: | 2         | 3        | 0.313012     | 0.768332          | 0.767985            |
PROGRESS: | 3         | 4        | 0.443015     | 0.768747          | 0.767712            |
PROGRESS: | 4         | 5        |

In [54]:
selected_words_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+-------------------+-----+-------+------+
 |    threshold     |        fpr        | tpr |   p   |  n   |
 +------------------+-------------------+-----+-------+------+
 |       0.0        | 0.000115406809002 | 0.0 | 28010 | 8665 |
 | 0.0010000000475  |   0.999884593191  | 1.0 | 28010 | 8665 |
 | 0.00200000009499 |   0.999884593191  | 1.0 | 28010 | 8665 |
 | 0.00300000002608 |   0.999769186382  | 1.0 | 28010 | 8665 |
 | 0.00400000018999 |   0.999769186382  | 1.0 | 28010 | 8665 |
 | 0.00499999988824 |   0.999769186382  | 1.0 | 28010 | 8665 |
 | 0.00600000005215 |   0.999769186382  | 1.0 | 28010 | 8665 |
 | 0.00700000021607 |   0.999769186382  | 1.0 | 28010 | 8665 |
 | 0.00800000037998 |   0.999769186382  | 1.0 | 28010 | 8665 |
 | 0.00899999961257 |   0.999769186382  | 1.0 | 28010 | 8665 |
 +------------------+-------------------+-----+-------+------+
 [1001 rows x 

In [55]:
selected_words_model.show(view='Evaluation')

In [41]:
selected_words_model["coefficients"]

name,index,class,value
(intercept),None,1,0.899735119511
awesome,None,1,1.00625649388
great,None,1,0.699683829426
fantastic,None,1,0.699443114061
amazing,None,1,0.892025920947
love,None,1,1.15720106058
horrible,None,1,-1.75774666269
bad,None,1,-0.904090478755
terrible,None,1,-1.73568678121
awful,None,1,-1.43812417008


In [31]:
diaper_champ_reviews = products[products['name'] == "Baby Trend Diaper Champ"]

In [34]:
len(diaper_champ_reviews)

333

In [42]:
diaper_champ_reviews['rating'].show(view='Categorical')

In [50]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

In [51]:
diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'all': 1L, 'bags.': 1L,'son,': 1L, '(i': 1L, ...",1,0
Baby Trend Diaper Champ,"This is absolutely, byfar, the best diaper ...",5.0,"{'just': 3L, 'money': 1L,'not': 2L, 'mechanism': ...",1,0
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'all': 1L, 'less': 1L,""friend's"": 1L, '(whi ...",1,0
Baby Trend Diaper Champ,I wanted a diaper pailthat didn't stink. I'd ...,5.0,"{'bags.': 1L, 'just': 1L,'less': 1L, 'sceptical': ...",1,0
Baby Trend Diaper Champ,I don't know how wesurvived the first year ...,5.0,"{'lysol': 1L, 'all': 1L,'just': 1L, ""don't"": 3L, ...",1,0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'just': 1L, 'when': 1L,'both': 1L, 'results': ...",1,0
Baby Trend Diaper Champ,The Diaper Champ wasrecommended to us in the ...,4.0,"{'lysol': 1L, 'open-':1L, 'all': 2L, 'hand!': ...",1,0
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'just': 1L, 'over': 1L,'rweek': 1L, 'sooo': 1L, ...",1,0
Baby Trend Diaper Champ,Wow! This is fabulous.It was a toss-up between ...,5.0,"{'and': 4L, '""genie"".':1L, 'since': 1L, ...",1,0
Baby Trend Diaper Champ,"As a first time mother, Iwanted to get the best ...",5.0,"{'and': 2L, 'bag!!!when':1L, 'love': 1L, 'figu ...",1,0


In [70]:
sentiment_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.9989298176718643]

In [71]:
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.7108950665616598]

In [59]:
diaper_champ_reviews[0]

{'amazing': 0.0,
 'awesome': 0L,
 'awful': 0L,
 'bad': 0.0,
 'fantastic': 0.0,
 'great': 0.0,
 'hate': 0L,
 'horrible': 0L,
 'love': 0.0,
 'name': 'Baby Trend Diaper Champ',
 'predicted_sentiment': 0.9989298176718643,
 'rating': 4.0,
 'review': "Ok - newsflash.  Diapers are just smelly.  We've had this pail for 2.5 years now.  It was our first and primary one.  There were no major smell problems until after one year, when our son started eating solids.  Also, we change the bag twice weekly as 3 days is about the max for smell-containment.  Around 20-22 months we started shopping for a container that would be less smelly and didn't find one as good.  (We have a cheaper one upstairs which broke immediately and always stunk!)  We finally just put the Diaper Champ in the attic a few months ago and use the cheap one with the flip-up lid - mainly since the cheapo fits inside the cabinet and we didn't notice a big difference in smell-control.  (The most helpful action is to tie the dirty diap